### 6週目の3日目へようこそ！

より多くのMCPサーバーを試してみましょう

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

###最初のタイプのMCPサーバー：ローカルで実行され、すべてローカル

ここに本当に興味深いものがあります：知識とグラフベースの記憶。

エンティティの永続的なメモリストア、それらについての観察、およびそれらの間の関係です。

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [ ]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/ed.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

### トレースを確認してください：

https://platform.openai.com/traces

### MCPサーバーの2番目のタイプ - ローカルで実行され、Webサービスを呼び出します

### Brave Search-謝罪 - これには別のAPIキーが必要です！しかし、それはまた無料です。

https://brave.com/search/api/

アカウントを設定し、 `brave_api_key`の下にキーを.envに入れます

In [ ]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### いつものように、トレースをチェックしてください。

https://platform.openai.com/traces

##そして今、3番目のタイプ：リモートで実行されます

実際には、「リモートMCPサーバー」別名「MCPサーバー」をホストした「マネージドMCPサーバー」を見つけるのは非常に困難です。

これは、MCPサーバーを使用または共有するための一般的なモデルではなく、リモートMCPサーバーを発見する標準的な方法はありません。

人類はいくつかのリモートMCPサーバーをリストしていますが、これらはビジネスユーザーとの有料アプリケーション用です。

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlareには、独自のリモートMCPサーバーを作成および展開するためのツールがありますが、これは一般的な慣行ではないようです。

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# 2番目のタイプに戻る：polygon.io mcpサーバー

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/stop.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">読んでください!!  -  </h2>
            <SPAN STYLE = "COLOR：＃FF7800;">金融市場データのこのサービスには、無料プランと有料プランの両方があり、食欲によって使用できます。
            </span>
        </td>
    </tr>
</table>

## 新しいセクション：polygon.ioの紹介

Polygon.ioは非常に人気のある金融データプロバイダーです。無料プランと有給計画があります。また、MCPサーバーもあります！

まず、価格を見るなど、優れたウェブサイトでPolygon.ioを読んでください。

https://polygon.io

### polygon.ioパート1：polygon.io無料サービス（もちろん、有料は完全にオプションになります！）

1。polygon.io（右上）にサインアップしてください  
2。サインインしたら、左側のナビゲーションで「キー」を選択してください
3.青い「新しいキー」ボタンを押します
4.キー名をコピーします
5。.envファイルを編集して行を追加します。

`polygon_api_key = xxxx`

In [ ]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("polygon_api_keyは設定されていません")

In [ ]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

###は、1日の終わりの価格をキャッシュするPythonモジュールに包まれています

このAPIを使用して株価を検索するPythonモジュール「Market.py`」を作成しました。

しかし、無料のAPIは非常に大きく制限されています。したがって、私は少し卑劣でした。株価を要求すると、この機能は日中の株式市場全体を取得し、データベースでキャッシュします。


In [ ]:
from market import get_share_price
get_share_price("AAPL")

In [ ]:
# 懸念を制限する料金はありません！

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

### そして、私はこれをMCPサーバーにしました

accounts.pyで行ったように。 `market_server.py`を参照してください

In [ ]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

### 試してみましょう！

うまくいけば、GPT-4o-MiniはAppleのシンボルがAAPLであることを知るのに十分賢いです

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## polygon.ioパート2：有料プラン - 完全にオプション！

興味がある場合は、毎月の計画を購読して、最新の市場データと無制限のAPI呼び出しを取得できます。

これを希望する場合は、Polygon.ioがリリースした完全なMCPサーバーを使用して、すべての機能を活用することも理にかなっています。


In [ ]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools


### うわー、それはたくさんのツールです！

それらを試してみましょう - うまくいけば、膨大な数のツールがGPT-4O-MINIを圧倒しません！

毎月29ドルのプランでは、APIの一部にアクセスできないため、どのAPIを呼び出すことができるかを指定する必要がありました。

あなたがより大きな計画に跳ね返った場合は、私の追加の制約を削除してください。

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## .envファイルをセットアップします

有料プランを使用することにした場合は、これを.envファイルに追加して、次のことを示してください。

`polygon_plan = paid`

そして、リアルタイムAPIのためにずっと行くことにした場合は、次のようにしてください。

`polygon_plan = realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("有料ポリゴンプランを購読することを選択したので、コードは15分の遅延で価格を調べます")
elif is_realtime_polygon:
    print("Wowzer-リアルタイムポリゴンプランを購読することを選択したので、コードはリアルタイム価格を調べます")
else:
    print(".ENVファイルによると、無料のポリゴンプランを購読することを選択したため、コードはEOD価格を調べます")

## そして、それは今日のためです！

「金融データセット」MCPサーバーを使用するこのラボの一部を削除しました。これは劣っているためです。APIが少なく、より高価です。

このようにして、同じプロバイダーを無料および有料APIで使用することができます。

ただし、コードを表示したい場合は、以前のバージョンについてはGit履歴を確認してください。

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/exercise.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">演習</h2>
            <SPAN STYLE = "Color：＃FF7800;"> MCPサーバーマーケットプレイスを探索し、3つのアプローチすべてを使用して独自の統合を行います。
            </span>
        </td>
    </tr>
</table>